# Azure ML training 

In [ ]:
import os
import pandas as pd

from azure.ai.ml import MLClient
from azure.ai.ml import command, Input, Output
from azure.ai.ml.entities import Data
from azure.ai.ml.entities import Model
from azure.ai.ml.entities import Environment, BuildContext
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential

from dotenv import load_dotenv
load_dotenv(override=True)

### Authentication
👉 
##### [Azure Machine Learning のリソースとワークフローの認証を設定する](https://learn.microsoft.com/ja-jp/azure/machine-learning/how-to-setup-authentication?view=azureml-api-2&tabs=sdk)

<h5> Azure ML Client の設定 </h5>
環境変数からAzureの認証情報を取得し、MLClientを初期化します。

In [ ]:
subscription_id = os.getenv("SUBSCRIPTION_ID")
resource_group = os.getenv("RESOURCE_GROUP")
workspace = os.getenv("AML_WORKSPACE_NAME")
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

### Register Data
👉 
##### [データ資産の作成と管理](https://learn.microsoft.com/ja-jp/azure/machine-learning/how-to-create-data-assets?view=azureml-api-2&tabs=python)

<h5> データセットの登録 </h5>
CSVファイルをAzure ML にデータセットとして登録します。

In [ ]:
data_uri = "./diabetes_data.csv"

data = Data(
    path = data_uri,
#    type = AssetTypes.URI_FOLDER,
    type = AssetTypes.URI_FILE,
    description = "test for demo",
    name = "diabetes_data_01",
    version = '1'
)

ml_client.data.create_or_update(data)

### Environment
👉
##### [CLI & SDK (v2) を使用して Azure Machine Learning 環境を管理する](https://learn.microsoft.com/ja-jp/azure/machine-learning/how-to-manage-environments-v2?view=azureml-api-2&tabs=python)

need to wait for the image build to complete

<h5> カスタム環境の作成</h5>
Dockerイメージとcondaファイルを使用してカスタム実行環境を作成します。

In [ ]:
env_docker_image = Environment(
    name="diabetes-env-01",
    image= "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file = "./environments/diabetes-env-01.yml",
    description="test for diabetes",
    version = "1"
    )
ml_client.environments.create_or_update(env_docker_image)

## Job
👉
##### [Azure Machine Learning CLI、SDK、REST API を使用してモデルをトレーニングする](https://learn.microsoft.com/ja-jp/azure/machine-learning/how-to-train-model?view=azureml-api-2&tabs=python)

<h5>トレーニングジョブの実行（カスタム環境使用）</h5>
カスタム環境とカスタム出力パスを使用してモデルのトレーニングジョブを実行します。

In [ ]:
job = command(
    experiment_name="train-diabetes-oh4ml-001",
    code="./scripts",
    command="python train-diabetes.py \
        --input_data ${{inputs.diabetes_data}} \
        --output_dir ${{outputs.output_dir}} \
        --alpha ${{inputs.alpha}}", 
    inputs={
        "diabetes_data": Input(
            type=AssetTypes.URI_FILE,
            path="diabetes_data_01@latest",
        ),
        "alpha": 0.03,
    },
    outputs={
        "output_dir": Output(     
            type="uri_folder",
            path="azureml://datastores/workspaceblobstore/paths/diabetes_oh4ml-test/",
            mode="mount"
        )
    },
    environment="diabetes-env-01@latest",
    compute="demo-cpucluster1"
)

# submit the command
returned_job = ml_client.create_or_update(job)
# get a URL for the status of the job
returned_job.services["Studio"].endpoint

## Job #2
👉
##### Use Default output folder and  Curated Environment

<h5>トレーニングジョブの実行（キュレート環境使用）</h5>
デフォルトの出力フォルダーとキュレートされた環境を使用してジョブを実行します。

In [ ]:
job = command(
    experiment_name="train-diabetes-oh4ml-001",
    code="./scripts",
    command="python train-diabetes.py \
        --input_data ${{inputs.diabetes_data}} \
        --alpha ${{inputs.alpha}}",
    inputs={
        "diabetes_data": Input(
            type=AssetTypes.URI_FILE,
            path="diabetes_data_01@latest",
        ),
        "alpha": 0.03,
    },
    environment="azureml://registries/azureml/environments/sklearn-1.5/labels/latest",
    compute="demo-cpucluster1"
)

# submit the command
returned_job = ml_client.create_or_update(job)
# get a URL for the status of the job
returned_job.services["Studio"].endpoint

### Register Model
👉

##### [Azure Machine Learning で登録済みモデルを操作する](https://learn.microsoft.com/ja-jp/azure/machine-learning/how-to-manage-models?view=azureml-api-2&tabs=python)

<h5>モデルの登録（ジョブ出力から）</h5>
実行済みのジョブの出力からトレーニング済みモデルを登録します。

In [ ]:
job_name = returned_job.name

file_model = Model(
#    path=f"azureml://jobs/{job_name}/outputs/paths/model/",
    path=f"runs:/{job_name}/outputs/",
    type=AssetTypes.MLFLOW_MODEL,
    name="diabetes_model_01",
    description="Model created from blob storage.",
)
ml_client.models.create_or_update(file_model)

##### Register from blob storage

<h5>モデルの登録（Blobストレージから）</h5>
Blobストレージに保存されたモデルを直接登録します。

In [ ]:
file_model = Model(
    path="azureml://datastores/workspaceblobstore/paths/diabetes_oh4ml-test/",
    type=AssetTypes.MLFLOW_MODEL,
    name="diabetes_model_test",
    description="Model created from blob storage.",
)
ml_client.models.create_or_update(file_model)